### Libraries, paths, and set-up

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import tensorflow as tf
import keras
import sys
from tensorflow.keras.layers import Layer, Dense, LSTM, GRU, Dropout, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# For reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Setting directory
os.chdir('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia')

# Custom Modules
from src.models.metrics import calculate_metrics


In [2]:
# Loading the windowed data
with open('data/processed/train_non_overlapping_windows.pkl', 'rb') as f:
    X_train_windows, Y_train_windows = pickle.load(f)
with open('data/processed/test_non_overlapping_windows.pkl', 'rb') as f:
    X_test_windows, Y_test_windows = pickle.load(f)

# Storing references for plant and agent names
train_references = [(window['plant'].iloc[0], window['agent'].iloc[0]) for window in X_train_windows]
test_references = [(window['plant'].iloc[0], window['agent'].iloc[0]) for window in X_test_windows]

# Dropping 'datetime', 'plant', and 'agent' columns and convert data types to float
for window in X_train_windows:
    window.drop(columns=['datetime', 'plant', 'agent'], errors='ignore', inplace=True)
    window.loc[:, :] = window.astype(float)  # Convert all remaining columns to float

for window in X_test_windows:
    window.drop(columns=['datetime', 'plant', 'agent'], errors='ignore', inplace=True)
    window.loc[:, :] = window.astype(float)  # Converting all remaining columns to float

# Converting to numpy arrays
X_train_np = np.array([window.values for window in X_train_windows])
Y_train_np = np.array([window for window in Y_train_windows])
X_test_np = np.array([window.values for window in X_test_windows])
Y_test_np = np.array([window for window in Y_test_windows])


In [3]:
# Checking data types and shapes
print("X_train_np dtype:", X_train_np.dtype)
print("Y_train_np dtype:", Y_train_np.dtype)
print("X_train_np shape:", X_train_np.shape)
print("Y_train_np shape:", Y_train_np.shape)

# Ensuring the data is of type float32
X_train_np = X_train_np.astype(np.float32)
Y_train_np = Y_train_np.astype(np.float32)
X_test_np = X_test_np.astype(np.float32)
Y_test_np = Y_test_np.astype(np.float32)

# Verifying the conversion
print("X_train_np dtype after conversion:", X_train_np.dtype)
print("Y_train_np dtype after conversion:", Y_train_np.dtype)


X_train_np dtype: object
Y_train_np dtype: float64
X_train_np shape: (126537, 24, 26)
Y_train_np shape: (126537, 24)
X_train_np dtype after conversion: float32
Y_train_np dtype after conversion: float32


### Ensuring GPU availability

In [12]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print()
print("GPU is", "AVAILABLE" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.14.0
Keras Version: 2.14.0

Python 3.9.13 (main, Oct 13 2022, 16:12:19) 
[Clang 12.0.0 ]

GPU is AVAILABLE


### T2V Layer

In [1]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

class Time2Vec(Layer):
    def __init__(self, kernel_size=1, **kwargs):
        self.kernel_size = kernel_size
        super(Time2Vec, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], self.kernel_size),
                                 initializer='uniform',
                                 trainable=True)
        self.B = self.add_weight(name='B',
                                 shape=(1, self.kernel_size),
                                 initializer='uniform',
                                 trainable=True)
        self.w = self.add_weight(name='w',
                                 shape=(input_shape[-1],),
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(name='b',
                                 shape=(input_shape[-1],),
                                 initializer='uniform',
                                 trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs):
        bias = self.w * inputs + self.b
        dp = tf.matmul(inputs, self.W) + self.B
        wgts = tf.math.sin(dp)
        return tf.concat([wgts, bias], -1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], 2 * input_shape[2])

### A model with hyperparameter options for tuning

In [6]:
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Dropout, Bidirectional, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
import keras_tuner as kt

def build_model(hp):
    inputs = Input(shape=(24, X_train_np.shape[2]))  # Assuming 24 hours and the preprocessed shape
    time2vec = Time2Vec(kernel_size=hp.Int('t2v_kernel_size', min_value=1, max_value=5))(inputs)
    
    x = time2vec
    
    for i in range(hp.Int('num_layers', 1, 3)):
        if hp.Choice('rnn_type', ['LSTM', 'GRU']) == 'LSTM':
            x = Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                                   return_sequences=True))(x)
        else:
            x = Bidirectional(GRU(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                                  return_sequences=True))(x)
        if hp.Boolean('dropout'):
            x = Dropout(rate=0.2)(x)

    # Adding Attention Mechanism
    attention = Attention()([x, x])
    
    # Reduce to single output
    x = Dense(1)(attention)
    
    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mse', 
                  metrics=['mae'])
    return model

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Bayesian Optimization tuner
tuner = kt.BayesianOptimization(build_model,
                                objective='val_mae',
                                max_trials=50,
                                directory='bayesian_optimization',
                                project_name='time2vec_rnn')

stop_early = EarlyStopping(monitor='val_loss', patience=5)

# Perform the hyperparameter search
tuner.search(X_train_np, Y_train_np, epochs=50, validation_data=(X_test_np, Y_test_np), callbacks=[stop_early])

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

Trial 50 Complete [00h 16m 50s]
val_mae: 6.864586353302002

Best val_mae So Far: 0.6937180161476135
Total elapsed time: 3d 06h 24m 44s


In [16]:
best_model.save('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/models/non_overlapping_NN')

INFO:tensorflow:Assets written to: /Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/models/non_overlapping_NN/assets


INFO:tensorflow:Assets written to: /Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/models/non_overlapping_NN/assets


In [14]:
tuner.get_best_hyperparameters()[0].values

{'t2v_kernel_size': 1,
 'num_layers': 1,
 'rnn_type': 'LSTM',
 'units': 128,
 'dropout': False,
 'learning_rate': 0.001}

### Make predictions and calculate metrics

In [4]:
from tensorflow.keras.models import load_model

best_model = load_model('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/models/non_overlapping_NN')

2024-05-28 15:22:09.998994: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-05-28 15:22:09.999041: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-28 15:22:09.999054: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-28 15:22:09.999092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-28 15:22:09.999109: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### Predictions on train dataset

In [8]:
y_pred = np.squeeze(best_model.predict(X_train_np), axis=-1)

3955/3955 [==============================] - 25s 6ms/step


In [9]:
calculate_metrics(Y_train_np, y_pred)

RMSE: 1.341751217842102
MAE: 0.24325740337371826
sMAPE(0-200): 0.11214802507311106%
R-squared: 0.9999781543700266


#### On test dataset

In [10]:
y_pred_test = np.squeeze(best_model.predict(X_test_np), axis=-1)

404/404 [==============================] - 3s 7ms/step


In [11]:
calculate_metrics(Y_test_np, y_pred_test)

RMSE: 4.621414661407471
MAE: 0.6937179565429688
sMAPE(0-200): 0.08409047732129693%
R-squared: 0.9999167259472834
